In [16]:
# ### basic required packages ###
!pip install numpy
!pip install pyopenms
!pip install biopython
!pip install openpyxl

In [17]:
# ### GOOGLE AUTH ###
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
!pip3 install --upgrade oauth2client 
!pip3 install --upgrade oauth2client 

In [18]:
#######################
###  Запускать тут ####
#######################

import glob
import pandas as pd
import os
import unicodedata

# # CHANGE THIS
# EXPERIMENT_NAME = "KJE0006"

# Список для хранения обработанных DataFrame из каждого файла


def xlsx_paths(EXPERIMENT_NAME):
    # specifying the path to csv files
    path = f"./requests/{EXPERIMENT_NAME}"

    # csv files in the path
    file_paths = glob.glob(path + "/*.xlsx")
    return file_paths

def csv_paths(EXPERIMENT_NAME):
    # specifying the path to csv files
    path = f"./requests/{EXPERIMENT_NAME}"

    txt_filepaths = glob.glob(path + "/*.txt")
    print(txt_filepaths)
    
    for path in txt_filepaths:
        os.replace(path, path.replace(".txt", ".csv"))
    
    # csv files in the path
    file_paths = glob.glob(path + "/*.csv")
    
    return file_paths

def is_valid_sequence(seq):

    if pd.isna(seq):
        return False
    valid_chars = set("ATGCUYRSWKMBDHVN.-")
    return all(c in valid_chars for c in str(seq))

def process_excel_files(xlsx_paths, dfs_list):

    for file_path in xlsx_paths:
        import numpy as np
        # Чтение файла Excel
        df = pd.read_excel(file_path, header=None)


        if len(df.columns)==1:
            df = df[0].str.split(',', expand=True)

        if len(df.columns)<4:
        #     return print("ERROR IN INPUT FILE")
            pass
        df.dropna(how='any', subset=(0,2,3), inplace=True)

        df[1].fillna("DMT OFF", inplace = True)

        # df curation
        for column in df.columns:
            df[column] = df[column].apply(lambda x: unicodedata.normalize('NFC', str(x)))
        #  pandas' Series.str.strip() method
        df = df.apply(lambda x: x.str.strip())

        df[0] = df[0].apply(lambda x: x.upper())

        # Получение имени самого левого столбца
        leftmost_column = df.columns[0]
        print(file_path, len(df))
        # Фильтрация строк согласно заданным условиям
        df = df[df[leftmost_column].apply(is_valid_sequence)]

        print(file_path, len(df))
        # Добавление обработанного DataFrame в список
        dfs_list.append(df)
        
        if dfs_list != None:
            pass
        else:
            dfs_list = []
            print([])
        
    return dfs_list
        
    
def process_csv_files(csv_paths, dfs_list):

    for file_path in csv_paths:
        # Чтение файла csv
        df = pd.read_csv(file_path, header=None, sep=",")
        print(df)
        df.dropna(how='any', inplace=True)
        if len(df.columns)==1:
            df = df[0].str.split(',', expand=True)
            
        if len(df.columns)==1:
            df = df[0].str.split(';', expand=True)

        if len(df.columns)<4:
            return print("ERROR IN INPUT FILE")

        # df curation
        for column in df.columns:
            df[column] = df[column].apply(lambda x: unicodedata.normalize('NFC', str(x)))
        #  pandas' Series.str.strip() method
        df = df.apply(lambda x: x.str.strip())
        
        df[0] = df[0].apply(lambda x: x.upper())

        # Получение имени самого левого столбца
        leftmost_column = df.columns[0]
        print(file_path, len(df))
        # Фильтрация строк согласно заданным условиям
        df = df[df[leftmost_column].apply(is_valid_sequence)]
        
        # Добавление обработанного DataFrame в список
        dfs_list.append(df)
        
        if dfs_list != None:
            pass
        else:
            dfs_list = []
            print([])
            
    return dfs_list
        
def process_files(EXPERIMENT_NAME):
    dfs_list = []
    dfs_list = process_excel_files(xlsx_paths(EXPERIMENT_NAME), dfs_list)
    if dfs_list != None:
        print(dfs_list)
    else:
        dfs_list = []
    dfs_list = process_csv_files(csv_paths(EXPERIMENT_NAME), dfs_list)
    return dfs_list

# EXPERIMENT_NAME = input()
# dfs_list=process_files(EXPERIMENT_NAME)



In [19]:
# EXPERIMENT_NAME = input()
# dfs_list=process_csv_files(csv_paths(EXPERIMENT_NAME), dfs_list)

# dfs_list = []
# dfs_list = process_csv_files(csv_paths(EXPERIMENT_NAME), dfs_list)

# dfs_list = []
# dfs_list = process_excel_files(xlsx_paths(EXPERIMENT_NAME), dfs_list)

In [20]:
# file_path="./requests/KJE0017/ФОРМА ЗАКАЗА - Данис Харрасов.xlsx"


# df

In [21]:
# # process_excel_files(xlsx_paths(EXPERIMENT_NAME), dfs_list)
# from google.colab import data_table
# data_table.enable_dataframe_formatter()

# from IPython.display import display

# # print the DataFrame
# display(dfs_list[-2][10:])

# Nikiteev_df=dfs_list[-2]

# Nikiteev_df=Nikiteev_df.dropna(how='any', inplace=True)

# display(Nikiteev_df[10:59])


In [22]:

def export_to_input_files(dfs_list, EXPERIMENT_NAME):
    # Конкатенация всех обработанных DataFrame по строкам
    final_df = None
    final_df = pd.concat(dfs_list, axis=0, join='outer', ignore_index=True)
    
    if (len(final_df)>384 and len(final_df)<=768):

        df_A = final_df[:384]
        df_B = final_df[384:]

        # Сохранение результата в новый файл CSV
        df_A.to_csv(f"./result_input/{EXPERIMENT_NAME}/{EXPERIMENT_NAME}_concatenated_file_plate_A.csv", index=False, sep = ',', header = None, encoding = "UTF-8")
        # Сохранение результата в новый файл Excel
        df_A.to_excel(f"./result_input/{EXPERIMENT_NAME}/{EXPERIMENT_NAME}_concatenated_file_plate_A.xlsx", index=False, header = None, encoding = "UTF-8")

         # Сохранение результата в новый файл CSV
        df_B.to_csv(f"./result_input/{EXPERIMENT_NAME}/{EXPERIMENT_NAME}_concatenated_file_plate_B.csv", index=False, sep = ',', header = None, encoding = "UTF-8")
        # Сохранение результата в новый файл Excel
        df_B.to_excel(f"./result_input/{EXPERIMENT_NAME}/{EXPERIMENT_NAME}_concatenated_file_plate_B.xlsx", index=False, header = None, encoding = "UTF-8")

    elif len(final_df)>768:
    
        print("error, too much oligos HALP")
    
    else:

        # Сохранение результата в новый файл CSV
        final_df.to_csv(f"./result_input/{EXPERIMENT_NAME}/{EXPERIMENT_NAME}_concatenated_file.csv", index=False, sep = ',', header = None, encoding = "UTF-8")

        # Сохранение результата в новый файл Excel
        final_df.to_excel(f"./result_input/{EXPERIMENT_NAME}/{EXPERIMENT_NAME}_concatenated_file.xlsx", index=False, header = None, encoding = "UTF-8")

In [23]:
#!/usr/bin/env python
#
# File: DrOligoInputPreparation_withFormsAPI.py
# Author: Oleg Fedorov, Kamil Zaynullin
#
# Copyright (C) 2023 Group of automated synthesis
#
# This file is part of gene synthesis automatization pipeline implemented in SBM.

from __future__ import print_function

import io
import os

########################
## GOOGLE API IMPORTS ##
########################
from apiclient import discovery
from google.oauth2.credentials import Credentials
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload
from httplib2 import Http
from oauth2client import client, file, tools

SCOPES = "https://www.googleapis.com/auth/drive"
DISCOVERY_DOC = "https://forms.googleapis.com/$discovery/rest?version=v1"


def creating_dirs(EXPERIMENT_NAME):
    #################
    ## CREATE DIR  ##
    #################
    cwd = os.getcwd()
    dir = os.path.join(cwd, "requests", EXPERIMENT_NAME)
    if not os.path.exists(dir):
        os.mkdir(dir)
        
    dir = os.path.join(cwd, "result_input", EXPERIMENT_NAME)
    if not os.path.exists(dir):
        os.mkdir(dir)


def google_auth():
    #################
    ## GOOGLE AUTH ##
    #################

    """
    caution! it appeares to be deprecated!!!

    """
    cwd = os.getcwd()
    token_file = os.path.join(cwd, "token.json")
    if os.path.exists(token_file):
        print("auth token already exists")
        store = file.Storage(token_file)
        creds = store.get()
    else:
        store = file.Storage("token.json")
        creds = None
        if not creds:
            flow = client.flow_from_clientsecrets("credentials.json", SCOPES)

            creds = tools.run_flow(flow, store)
    return creds


def retrive_forms_data(creds):
    ##################
    ## GOOGLE FORMS ##
    ##################
    service = discovery.build(
        "forms",
        "v1",
        http=creds.authorize(Http()),
        discoveryServiceUrl=DISCOVERY_DOC,
        static_discovery=False,
    )

    form_id = "1bVGyD-pz5w3BXM48yHWnhEDVnYNVgzRrF3ysvjAbqHQ"
    result = service.forms().responses().list(formId=form_id).execute()
    file_list = [
        response["answers"]["778e4a4f"]["fileUploadAnswers"]["answers"]
        for response in result["responses"]
    ]
    print(file_list)
    return file_list


####################################################
# downloading from Google Disk and writing to file #
####################################################
def download_request_file(fileID, filename, creds, EXPERIMENT_NAME):
    try:
        service = discovery.build("drive", "v3", credentials=creds)

        file_id = fileID

        # pylint: disable=maybe-no-member
        request = service.files().get_media(fileId=file_id)
        file = io.BytesIO()
        downloader = MediaIoBaseDownload(file, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
            print(f"Download {int(status.progress() * 100)}.")

    except HttpError as error:
        print(f"An error occurred: {error}")
        file = None
    filepath = os.path.join(os.getcwd(), "requests", EXPERIMENT_NAME, filename)
    with open(filepath, "wb") as outfile:
        # Copy the BytesIO stream to the output file
        outfile.write(file.getbuffer())

    return print(file.getvalue())


def download_all_files(creds, EXPERIMENT_NAME):
    for file in retrive_forms_data(creds):
        download_request_file(file[0]["fileId"], file[0]["fileName"], creds, EXPERIMENT_NAME)


##########
## main ##
##########
def main() -> None:
    EXPERIMENT_NAME = input("enter EXPERIMENT_NAME:")
    creating_dirs(EXPERIMENT_NAME)
    # retrive_forms_data()
    creds = google_auth()
    download_all_files(creds, EXPERIMENT_NAME)
    print("downloads finished")
    return EXPERIMENT_NAME

In [24]:
#####################################
#### EXPORTING EMAILS FROM FORMS ####
#####################################

import re  
import json

def get_emails_data():
    creds = google_auth()

    service = discovery.build(
        "forms",
        "v1",
        http=creds.authorize(Http()),
        discoveryServiceUrl=DISCOVERY_DOC,
        static_discovery=False,
    )

    form_id = "1bVGyD-pz5w3BXM48yHWnhEDVnYNVgzRrF3ysvjAbqHQ"
    result = service.forms().responses().list(formId=form_id).execute()
#     print(json.dumps(result, indent=2))
    
    return result

def remove_duplicate_emails(response_str):
    response_dict = json.loads(response_str) # Преобразование JSON-строки в Python-объект
    email_set = set()

    for response in response_dict['responses']:
        email = response.get('respondentEmail', None)
        if email:
            email_set.add(email)

        email_set.add(response['answers']['4d5a72c8']['textAnswers']['answers'][0]['value'])

    email_list = list(email_set)

#     print(email_list)
    return email_list

regex = re.compile(r'([A-Za-z0-9]+[.-_])*[A-Za-z0-9\-]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+')  
  
def emailValid(unique_emails):
    iterable = unique_emails.copy()
    for item in iterable:
#         print(item)
        if re.fullmatch(regex, item):
            pass
            print(item, "The given mail is valid")  
        else:
            unique_emails.remove(item)
            print(item, "The given mail is invalid")  
#     print(len(iterable))
    return unique_emails

def create_file_with_emails(EXPERIMENT_NAME):
    # Использования функции
    result = get_emails_data()
    response_str = json.dumps(result, indent=2)
    unique_emails = remove_duplicate_emails(response_str)
#     print(unique_emails)
    unique_emails = emailValid(unique_emails)
#     print(unique_emails)
    
    # write to file
    with open(f"./result_input/{EXPERIMENT_NAME}/{EXPERIMENT_NAME}_customer_emails.txt", 'w') as f:
        for email in unique_emails:
            f.write(f"{email}\n")


In [25]:
def combined_pipeline():
    EXPERIMENT_NAME = main()
    print(EXPERIMENT_NAME)
    dfs_list=process_files(EXPERIMENT_NAME)
    print(len(dfs_list))
    export_to_input_files(dfs_list, EXPERIMENT_NAME)
    create_file_with_emails(EXPERIMENT_NAME)
    return dfs_list
dfs_list = combined_pipeline()

enter EXPERIMENT_NAME:KJE0031
auth token already exists
[[{'fileId': '1u90OWdEBsRlQ4urhEYpVaHdBjtdqjjaN', 'fileName': 'Дополнительный_рациональный_дизайн - Юлия Кузьмина.xlsx', 'mimeType': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'}], [{'fileId': '1RYGaHbVBAyXSi5aIJ7-kLZxFm5sarbLO', 'fileName': '2024.04.05_oligos_Alekhina - Olga Alekhina.xlsx', 'mimeType': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'}], [{'fileId': '1PofTyt1KQwaWAfY-5lAQOzoBE6tDuJ_e', 'fileName': 'Олиги разное и фх174 15-04-2024 - Глеб Фисунов.txt', 'mimeType': 'text/plain'}], [{'fileId': '1v3hdfZrhmhl8XPgW10onnvguWE5CaXI7', 'fileName': 'PAM CAS - Юлия Кузьмина.xlsx', 'mimeType': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'}], [{'fileId': '1koSVRsv1xWOUGwotRWRMHqYViKUV9nha', 'fileName': 'Калибровочные буквы - Юлия Кузьмина.xlsx', 'mimeType': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'}], [{'fileId': '1HRKZFMNcJOoyhY37Yp8fjCnM

Download 100.
b'PK\x03\x04\x14\x00\x06\x00\x08\x00\x00\x00!\x00b\xee\x9dh^\x01\x00\x00\x90\x04\x00\x00\x13\x00\x08\x02[Content_Types].xml \xa2\x04\x02(\xa0\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

Download 100.
b'CGACGGGCCCGGGATCCGATGAGTTTTATCGCTTCCATGAC,DMT ON,phiX174-V2_F1_P-F_(pTZ/F),FISUNOV\r\nCCTCGCGAATGCATCTAGATCCACGGCGCTTTAAAATAGT,DMT ON,phiX174-V2_F1_P-R_(pTZ/F),FISUNOV\r\nCGACGGGCCCGGGATCCGATACTATTTTAAAGCGCCGTGG,DMT ON,phiX174-V2_F2_P-F_(pTZ/F),FISUNOV\r\nCCTCGCGAATGCATCTAGATCACACAGTCCTTGACGGTA,DMT ON,phiX174-V2_F2_P-R_(pTZ/F),FISUNOV\r\nCGACGGGCCCGGGATCCGATTACCGTCAAGGACTGTGTG,DMT ON,phiX174-V2_F3_P-F_(pTZ/F),FISUNOV\r\nCCTCGCGAATGCATCTAGATATGCGGTTATCCATCTGCT,DMT ON,phiX174-V2_F3_P-R_(pTZ/F),FISUNOV\r\nCGACGGGCCCGGGATCCGATAGCAGATGGATAACCGCAT,DMT ON,phiX174-V2_F4_P-F_(pTZ/F),FISUNOV\r\nCCTCGCGAATGCATCTAGATTGCAGGTTGGATACGC,DMT ON,phiX174-V2_F4_P-R_(pTZ/F),FISUNOV\r\nGATAAAAATGATTGGCGTATCCAACCTGCAGAGTTTTATCGCTTCCATGAC,DMT ON,phiX174-V2_F1_P-F_T,FISUNOV\r\nGCCTCGGTACGGTCAGGCATCCACGGCGCTTTAAAATAGT,DMT ON,phiX174-V2_F1_P-R_T,FISUNOV\r\nTTATTTGAATATCTATAACAACTATTTTAAAGCGCCGTGG,DMT ON,phiX174-V2_F2_P-F_T,FISUNOV\r\nACGGGGAAGGACGTCAATAGTCACACAGTCCTTGACGGTA,DMT ON,phiX174-V2_F2_P

Download 100.
b'PK\x03\x04\x14\x00\x06\x00\x08\x00\x00\x00!\x00b\xee\x9dh^\x01\x00\x00\x90\x04\x00\x00\x13\x00\x08\x02[Content_Types].xml \xa2\x04\x02(\xa0\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

Download 100.
b'GAGTTTTATCGCTTCCATGACGCAGAAGTTAACACTTTCGGATATTTCTGATGAG,DMT ON,phiX174-V2_F1-1,FISUNOV\r\nGCAGTAGTAATTCCTGCTTTATCAAGATAATTTTTCGACTCATCAGAAATATCCGAAAGTGT,DMT ON,phiX174-V2_F1-2,FISUNOV\r\nTGATAAAGCAGGAATTACTACTGCTTGTTTACGAATTAAATCGAAGTGGACTGCTGG,DMT ON,phiX174-V2_F1-3,FISUNOV\r\nAGGATAGGTCGAATTTTCTCATTTTCCGCCAGCAGTCCACTTCGATT,DMT ON,phiX174-V2_F1-4,FISUNOV\r\nAAAATGAGAAAATTCGACCTATCCTTGCGCAGCTCGAGAAGCTCTTACTTTGCGACCTTTCG,DMT ON,phiX174-V2_F1-5,FISUNOV\r\nTCAGTTTTTGACAGAATCGTTAGTTGATGGCGAAAGGTCGCAAAGTAAGAG,DMT ON,phiX174-V2_F1-6,FISUNOV\r\nACTAACGATTCTGTCAAAAACTGACGCGTTGGATGAGGAGAAGTGGCTTAATATG,DMT ON,phiX174-V2_F1-7,FISUNOV\r\nATCTAAACCAGTCCTTGACGAACGTGCCAAGCATATTAAGCCACTTCTCCTCATC,DMT ON,phiX174-V2_F1-8,FISUNOV\r\nTTCGTCAAGGACTGGTTTAGATATGAGTCACATTTTGTTCATGGT,DMT ON,phiX174-V2_F1-9,FISUNOV\r\nACGCTCTTTTAAAATGTCAACAAGAGAATCTCTACCATGAACAAAATGTGACTCAT,DMT ON,phiX174-V2_F1-10,FISUNOV\r\nCTTGTTGACATTTTAAAAGAGCGTGGATTACTATCTGAGTCCGATGCTGTTC,DMT ON,phiX174-V2_F1-11,FISUNOV\r\n

...

Download 100.
b'ATGTCAGGTCGTAATGTGGAGTCACATATGGAGCGTAACGAAAAAGTAGTG,DMT ON,CAHS94063_F0-1,CUSTOMER\r\nATCTGCATGACCAGAATTATTAACCACTACTTTTTCGTTACGCT,DMT ON,CAHS94063_F0-2,CUSTOMER\r\nTTAATAATTCTGGTCATGCAGATGTTAAAAAACAACAACAACAAGTGGAACAC,DMT ON,CAHS94063_F0-3,CUSTOMER\r\nCTTCAGTGTGAGTAAATTCTGTGTGTTCCACTTGTTGTTGT,DMT ON,CAHS94063_F0-4,CUSTOMER\r\nACAGAATTTACTCACACTGAAGTGAAGGCTCCTTTAATCCATCC,DMT ON,CAHS94063_F0-5,CUSTOMER\r\nCCCCTGTTGAAATGATTGGTGGTGCAGGATGGATTAAAGGAGCCT,DMT ON,CAHS94063_F0-6,CUSTOMER\r\nCCAATCATTTCAACAGGGGCTGCTGGTTTAGCTGAAGAAAT,DMT ON,CAHS94063_F0-7,CUSTOMER\r\nAGCAGTAAATCCTTGTCCAACAATTTCTTCAGCTAAACCAGC,DMT ON,CAHS94063_F0-8,CUSTOMER\r\nTGGACAAGGATTTACTGCTTCGGCAGCTAGAATTTCAGGTGGAACAG,DMT ON,CAHS94063_F0-9,CUSTOMER\r\nTAGCTGCTGATGGTTGTAAGTGAACTTCAGCTGTTCCACCTGAAATTCTAG,DMT ON,CAHS94063_F0-10,CUSTOMER\r\nTTACAACCATCAGCAGCTATGACTGAAGAGGCTAGACGTGACCAAGAACGTTATC,DMT ON,CAHS94063_F0-11,CUSTOMER\r\nCTTAGCAATAGATTCTTGTTCTTGACGATAACGTTCTTGGTCACG,DMT ON,CAHS94063_F0-12,CUSTOMER\r\nCA

Download 100.
b'PK\x03\x04\x14\x00\x08\x08\x08\x00\x80#\x8fX\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x18\x00\x00\x00xl/drawings/drawing1.xml\x9d\xd0]n\xc20\x0c\x07\xf0\x13\xec\x0eU\xdeiZ\x18\x13C\x14^\xd0N0\x0e\xe0%n\x1b\x91\x8f\xca\x0e\xa3\xdc~\xd1J6i{\x01\x1em\xcb?\xf9\xef\xcdnt\xb6\xf8Db\x13|#\xea\xb2\x12\x05z\x15\xb4\xf1]#\x0e\xefo\xb3\x95(8\x82\xd7`\x83\xc7F\\\x90\xc5n\xfb\xb4\x195\xad\xcf\xbc\xa7"\xed{^\xa7\xb2\x11}\x8c\xc3ZJV=:\xe02\x0c\xe8\xd3\xb4\r\xe4 \xa6\x92:\xa9\t\xceIvV\xce\xab\xeaE\xf2@\x08\x9a{\xc4\xb8\x9f&\xe2\xea\xc1\x03\x9a\x03\xe3\xf3\xfeM\xd7\x84\xb65\n\xf7A\x9d\x1c\xfa8!\x84\x16b\xfa\x05\xf7f\xe0\xac\xa9\x07\xaeQ=P\xfc\x01\xc6\x7f\x823\x8a\x02\x876\x96*\xb8\xeb)\xd9HB\xfd<\t8\xfe\x1a\xf5\xdd\xc8R\xbe\xca\xd5_\xc8\xdd\x14\xc7\x01\x1dO\xc3,\xb9Cz\xc8\x87\xb1&^\xbe\x93eFw\xee\x81\xb7h\x03\x1d\x81\xcb\xc8\xb88\xf8\xe3\xdd\xb1*\xc96\xb5(+l\xb1^\xde\xad\xcc\xb3"\xb7_PK\x07\x08\x07bi\x83\x05\x01\x00\x00\x07\x03\x00\x00PK\x03\x04\x14\x00\x08\x08\x08\x00\x80#\x8fX\

Download 100.
b'PK\x03\x04\x14\x00\x08\x08\x08\x00q;\x88X\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x18\x00\x00\x00xl/drawings/drawing1.xml\x9d\xd0]n\xc20\x0c\x07\xf0\x13\xec\x0eU\xdeiZ\x18\x13C\x14^\xd0N0\x0e\xe0%n\x1b\x91\x8f\xca\x0e\xa3\xdc~\xd1J6i{\x01\x1em\xcb?\xf9\xef\xcdnt\xb6\xf8Db\x13|#\xea\xb2\x12\x05z\x15\xb4\xf1]#\x0e\xefo\xb3\x95(8\x82\xd7`\x83\xc7F\\\x90\xc5n\xfb\xb4\x195\xad\xcf\xbc\xa7"\xed{^\xa7\xb2\x11}\x8c\xc3ZJV=:\xe02\x0c\xe8\xd3\xb4\r\xe4 \xa6\x92:\xa9\t\xceIvV\xce\xab\xeaE\xf2@\x08\x9a{\xc4\xb8\x9f&\xe2\xea\xc1\x03\x9a\x03\xe3\xf3\xfeM\xd7\x84\xb65\n\xf7A\x9d\x1c\xfa8!\x84\x16b\xfa\x05\xf7f\xe0\xac\xa9\x07\xaeQ=P\xfc\x01\xc6\x7f\x823\x8a\x02\x876\x96*\xb8\xeb)\xd9HB\xfd<\t8\xfe\x1a\xf5\xdd\xc8R\xbe\xca\xd5_\xc8\xdd\x14\xc7\x01\x1dO\xc3,\xb9Cz\xc8\x87\xb1&^\xbe\x93eFw\xee\x81\xb7h\x03\x1d\x81\xcb\xc8\xb88\xf8\xe3\xdd\xb1*\xc96\xb5(+l\xb1^\xde\xad\xcc\xb3"\xb7_PK\x07\x08\x07bi\x83\x05\x01\x00\x00\x07\x03\x00\x00PK\x03\x04\x14\x00\x08\x08\x08\x00q;\x88X\x00\x0

Download 100.
b'PK\x03\x04\x14\x00\x06\x00\x08\x00\x00\x00!\x00b\xee\x9dh^\x01\x00\x00\x90\x04\x00\x00\x13\x00\x08\x02[Content_Types].xml \xa2\x04\x02(\xa0\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

['./requests/KJE0031/Олиги разное и фх174 15-04-2024 - Глеб Фисунов.txt']
9
auth token already exists
alekhina.olga@gmail.com The given mail is valid
varvaramys@gmail.com The given mail is valid
v.shokina@sysbiomed.ru The given mail is valid
kuzminajk2000@gmail.com The given mail is valid
ikchudinov@gmail.com The given mail is valid
vvedenskiia@sysbiomed.ru The given mail is valid
vvedenskiia@gmail.com The given mail is valid
lukina.al98@gmail.com  The given mail is invalid
lukina.al98@gmail.com The given mail is valid
herr.romanoff@gmail.com The given mail is valid


In [ ]:
# !jupyter labextension install @jupyterlab/dataregistry-extension

In [26]:
len(dfs_list)

9

In [27]:
dfs_list

[                      0       1     2           3
 3   GTGCCAGCMGCCGCGGTAA  DMT ON  515F  VVEDENSKII
 4  GGACTACHVGGGTWTCTAAT  DMT ON  806R  VVEDENSKII
 5  AGAGTTTGATYMTGGCTCAG  DMT ON   27F  VVEDENSKII
 6    GCTGCCTCCCGTAGGAGT  DMT ON  338R  VVEDENSKII,
                                                     0       1  \
 0   AGGAGGTTTAAGATATGACTTCTAAGGTTTATGATCCTGAATTAAG...  DMT ON   
 1          ACTGAGGTCCTGTAATCATTCTTTTTCTTAATTCAGGATCAT  DMT ON   
 2           ATGATTACAGGACCTCAGTGGTGGGCAAGATGTAAACAAAT  DMT ON   
 3            ATAAATGAATCAAGCACGTTCATTTGTTTACATCTTGCCC  DMT ON   
 4   AACGTGCTTGATTCATTTATTAATTACTATGATTCTGAAAAACATG...  DMT ON   
 5     CATTACCATGTAAAAAAATCACAGCATTCTCAGCATGTTTTTCAGAA  DMT ON   
 6       TGTGATTTTTTTACATGGTAATGCTGCGTCATCATATTTATGGCG  DMT ON   
 7         GGTTCAACATGAGGAACTACGTGACGCCATAAATATGATGACG  DMT ON   
 8   TAGTTCCTCATGTTGAACCTGTTGCAAGATGTATCATCCCAGACCT...  DMT ON   
 9         CAGATTTACCTGATTTTCCCATACCAATTAGGTCTGGGATGAT  DMT ON   
 10  GGGAAAATCAGGT

In [38]:
EXPERIMENT_NAME = "KJE0031"
dfs_list=process_files(EXPERIMENT_NAME)

./requests/KJE0031/праймеры Введенский_10_04_24 - Андрей Введенский.xlsx 6
./requests/KJE0031/праймеры Введенский_10_04_24 - Андрей Введенский.xlsx 4
./requests/KJE0031/2024.04.05_oligos_Alekhina - Olga Alekhina.xlsx 60
./requests/KJE0031/2024.04.05_oligos_Alekhina - Olga Alekhina.xlsx 60
./requests/KJE0031/240415 Chudinov - Ivan Chudinov.xlsx 6
./requests/KJE0031/240415 Chudinov - Ivan Chudinov.xlsx 4
./requests/KJE0031/Дополнительный_рациональный_дизайн - Юлия Кузьмина.xlsx 16
./requests/KJE0031/Дополнительный_рациональный_дизайн - Юлия Кузьмина.xlsx 16
./requests/KJE0031/PAM CAS - Юлия Кузьмина.xlsx 12
./requests/KJE0031/PAM CAS - Юлия Кузьмина.xlsx 12
./requests/KJE0031/ФОРМА ЗАКАЗА - Varvara Shokina.xlsx 28
./requests/KJE0031/ФОРМА ЗАКАЗА - Varvara Shokina.xlsx 26
./requests/KJE0031/Калибровочные буквы - Юлия Кузьмина.xlsx 16
./requests/KJE0031/Калибровочные буквы - Юлия Кузьмина.xlsx 16
./requests/KJE0031/праймеры hMPV - Alexandra Lukina.xlsx 12
./requests/KJE0031/праймеры hMPV -

In [43]:
export_to_input_files(dfs_list, EXPERIMENT_NAME)

In [45]:
len(dfs_list)

12

In [47]:
# Использования функции
result = get_emails_data()
response_str = json.dumps(result, indent=2)
unique_emails = remove_duplicate_emails(response_str)
#     print(unique_emails)
unique_emails = emailValid(unique_emails)
#     print(unique_emails)

# write to file
with open(f"./result_input/{EXPERIMENT_NAME}/{EXPERIMENT_NAME}_customer_emails.txt", 'w') as f:
    for email in unique_emails:
        f.write(f"{email}\n")

auth token already exists
alekhina.olga@gmail.com The given mail is valid
varvaramys@gmail.com The given mail is valid
v.shokina@sysbiomed.ru The given mail is valid
kuzminajk2000@gmail.com The given mail is valid
ikchudinov@gmail.com The given mail is valid
vvedenskiia@sysbiomed.ru The given mail is valid
vvedenskiia@gmail.com The given mail is valid
lukina.al98@gmail.com  The given mail is invalid
lukina.al98@gmail.com The given mail is valid
herr.romanoff@gmail.com The given mail is valid
